<a href="https://colab.research.google.com/github/stelios191/Image-and-Video-Preprocessing-from-Scratch/blob/main/Image_and_Video_Preprocessing_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python


Libraries

In [ ]:
import os
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

Read and show the image

input the image that contains my name written in Arial, point 72, capital letters.

In [ ]:
im = cv2.imread("cw1.png")
plt.imshow(im)

Image Atrributes

In [ ]:
# Read in image
cw = cv2.imread("cw1.png", 1)

# print the size  of image
print("Image size (H, W, C) is:", cw.shape)

# print data-type of image
print("Data type of image is:", cw.dtype)

Write a function that takes as input an image I, rotates it by an angle θ1 and horizontally skews it by
an angle, θ2. Write the matrix formulation for image rotation R(.) and skewing S(.). Define all the
variables.

In [ ]:
def ICV_rotate(image, rotation_angle_degrees, fill_value=0):
    theta = math.radians(rotation_angle_degrees)
    # Compute the cosine and sine of the rotation angle
    cos_theta = math.cos(theta)
    sin_theta = math.sin(theta)
    #Original image dimensions
    orig_height, orig_width = image.shape[:2]
    orig_center_y, orig_center_x = orig_height // 2, orig_width // 2
    # Compute the bounding box of the new image
    new_width = int(abs(orig_height * sin_theta) + abs(orig_width * cos_theta))
    new_height = int(abs(orig_height * cos_theta) + abs(orig_width * sin_theta))
    # an empty image for the output with the new dimensions
    rotated_image = np.full((new_height, new_width, image.shape[2]), fill_value, dtype=image.dtype)
    #New image center
    new_center_x, new_center_y = new_width // 2, new_height // 2

    # Perform rotation
    for h in range(new_height):
        for w in range(new_width):
            # Apply the inverse rotation matrix to get the original coordinates
            x_orig = (cos_theta * (h - new_center_x) + sin_theta * (h - new_center_y)) + orig_center_x
            y_orig = (-sin_theta * (h - new_center_x) + cos_theta * (h - new_center_y)) + orig_center_y
            # Checks the coordinates if are within the boundaries
            if 0 <= x_orig < orig_width and 0 <= y_orig < orig_height:
                # Assign the pixel value from the original image to the rotated image
                rotated_image[h, w] = image[int(y_orig), int(x_orig)]
    return rotated_image

def ICV_skew(image, skew_angle_degrees, fill_value=0):
    skew_angle_radians = math.radians(skew_angle_degrees)
    skew_offset = abs(image.shape[0] * np.tan(skew_angle_radians))
    # Compute the new width to accommodate the skew
    new_width = int(image.shape[1] + skew_offset)
    # Create an empty image with the new dimensions
    skewed_image = np.full((image.shape[0], new_width, image.shape[2]), fill_value, dtype=image.dtype)
    # Skew transformation
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            # Apply the skew transformation to the x
            new_x = int(x + y * np.tan(skew_angle_radians))

            # Assign the original pixel value to the new coordinates in the skewed image
            skewed_image[y, new_x] = image[y, x]
    return skewed_image

Rotate clockwise
the image you created by 30, 60, 120 and -50 degrees. Skew the same image by 10, 40 and 60
degrees.

In [ ]:
# Rotate images
rotated_image_30 = ICV_rotate(cw, 30)
rotated_image_60 = ICV_rotate(cw, 60)
rotated_image_120 = ICV_rotate(cw, 120)
rotated_image_minus = ICV_rotate(cw, -50)

# Display the rotated images
plt.figure()
plt.title('Rotated by 30 degrees')
plt.imshow(rotated_image_30)
plt.show()

plt.figure()
plt.title('Rotated by 60 degrees')
plt.imshow(rotated_image_60)
plt.show()

plt.figure()
plt.title('Rotated by 120 degrees')
plt.imshow(rotated_image_120)
plt.show()

plt.figure()
plt.title('Rotated by -50 degrees')
plt.imshow(rotated_image_minus)
plt.show()

# Skew images
skewed_image_10 = ICV_skew(cw, 10)
skewed_image_40 = ICV_skew(cw, 40)
skewed_image_60 = ICV_skew(cw, 60)

# Display the skewed images
plt.figure()
plt.title('Skewed by 10 degrees')
plt.imshow(skewed_image_10)
plt.show()

plt.figure()
plt.title('Skewed by 40 degrees')
plt.imshow(skewed_image_40)
plt.show()

plt.figure()
plt.title('Skewed by 60 degrees')
plt.imshow(skewed_image_60)
plt.show()


Rotate the image by θ1 = 20 clockwise and then skew the result by θ2 = 50.

In [ ]:
# Rotate the image by θ1 = 20 degrees clockwise
rotated_image_20 = ICV_rotate(cw, 20)

# Skew the rotated image by θ2 = 50 degrees
skewed_image_50 = ICV_skew(rotated_image_20, 50)

# Display the rotated and skewed image
plt.figure()
plt.title('Rotated by 20 degrees and skewed by 50 degrees')
plt.imshow(skewed_image_50)
plt.show()

Skew the image by θ2 = 50 and then rotate the result by θ1 = 20 clockwise.


In [ ]:
# Skew the image by θ2 = 50 degrees
skewed_image = ICV_skew(cw, 50)

# Rotate the skewed image by θ1 = 20 degrees clockwise
rotated_image = ICV_rotate(skewed_image_50, 20)

# Display the skewed and rotated image
plt.figure()
plt.title('Skewed by 50 degrees and rotated by 20 degrees clockwise')
plt.imshow(rotated_image)
plt.show()

2) Convolution. (Use Dataset A)
Convolution provides a way of multiplying two arrays to produce a third array. Depending on the designed
filter and the intended effect, the kernel can be a matrix of dimensions, for example, 3x3, 5x5 or 7x7.


a) Code a function that takes an input image, performs convolution with a given kernel, and returns the
resulting image.

In [ ]:
def ICV_convolution(image: np.array, kernel: np.array, padding='valid'):
    image_height, image_width, num_channels = image.shape

    kernel_height, kernel_width = kernel.shape
    padding_height, padding_width = 0, 0

    if padding == 'same':
        padding_height = kernel_height // 2
        padding_width = kernel_width // 2
        padded_height = image_height + 2 * padding_height
        padded_width = image_width + 2 * padding_width
        padded_image = np.zeros((padded_height, padded_width, num_channels))
        padded_image[padding_height:-padding_height, padding_width:-padding_width, :] = image
    else:
        padded_image = image

    # Compute dimensions for the output image
    output_height = image_height + 2 * padding_height - kernel_height + 1
    output_width = image_width + 2 * padding_width - kernel_width + 1
    # Initialize the output image with zeros
    convolved_image = np.zeros((output_height, output_width, num_channels))

    # Perform convolution by sliding the kernel over the image
    for row in range(output_height):
        for col in range(output_width):
            for channel in range(num_channels):
                # Extract the corresponding image patch
                image_patch = padded_image[row:row+kernel_height, col:col+kernel_width, channel]
                convolved_image[row, col, channel] = np.sum(image_patch * kernel)
    # Clip pixel values
    convolved_image = np.clip(convolved_image, 0, 255)
    return convolved_image.astype(np.uint8)

 Design a convolution kernel that computes, for each pixel, the average intensity value in a 3x3 region.
Use this kernel and the filtering function above, and save the resulting image.

In [ ]:
average_kernel = np.ones((3, 3)) / 9
image = cv2.imread ('car-1.jpg')

# Perform convolution and save the output image.
output_image = ICV_convolution(image, average_kernel)
cv2.imwrite('average_intensity_image.png', output_image)

Use the kernels provided below, apply the filtering function and save the resulting images. Comment
on the effect of each kernel.
kernel A
1 2 1
2 4 2
1 2 1
kernel B
0 1 0
1 -4 1
0 1 0

In [ ]:
kernel_A = np.array([[1,2,1],
                     [2,4,2],
                     [1,2,1]])/9

kernel_B = np.array([[0,1,0],
                     [1,-4,1],
                     [0,1,0]])

# Apply convolution with kernel A
kernel_image_A = ICV_convolution(image, kernel_A)
cv2.imwrite('kernel_A.jpg', kernel_image_A)

# Apply convolution with kernel B
kernel_image_B = ICV_convolution(image, kernel_B)
cv2.imwrite('kernel_B.jpg', kernel_image_B)

d) Use the filtering function for the following filtering operations: (i) A followed by A; (ii) A followed by B;
(iii) B followed by A. Comment the results.

In [ ]:
# (i) A followed by A
kernel_AA = ICV_convolution(image, kernel_A)
kernel_AA = ICV_convolution(kernel_AA, kernel_A)
cv2.imwrite('kernel_AA.jpg', kernel_AA)

# (ii) A followed by B
kernel_AB = ICV_convolution(image, kernel_A)
kernel_AB = ICV_convolution(kernel_AB, kernel_B)
cv2.imwrite('kernel_AB.jpg', kernel_AB)

# (iii) B followed by A
kernel_BA = ICV_convolution(image, kernel_B)
kernel_BA = ICV_convolution(kernel_BA, kernel_A)
cv2.imwrite('kernel_BA.jpg', kernel_BA)

Video Segmentation. (Use Dataset B)
A colour histogram h(.) can be generated by counting how many times each colour occurs in an image.
Histogram intersection can be used to match a pair of histograms. Given a pair of histograms, e.g., of an
input image I and a model M, each containing n bins, the intersection of the histograms is defined as
∑ min[h(Ij), h(Mj)]
n
j=1
.

a) Write a histogram function that returns the colour histogram of an input image. Visualize the histogram
and save the corresponding figure. For a given video sequence, use the above function to construct
the histogram of each frame.


In [ ]:
def ICV_histogram(image):
    h_bins = 256
    hist = [np.zeros(h_bins), np.zeros(h_bins), np.zeros(h_bins)]
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            hist[0][image[i, j, 0]] += 1
            hist[1][image[i, j, 1]] += 1
            hist[2][image[i, j, 2]] += 1
    return hist

def ICV_histogram_with_frame(frame, hist, frame_idx, filename=None):
    plt.figure(figsize=(12, 6))

    # Plot the frame
    plt.subplot(1, 2, 1)
    plt.imshow(frame)
    plt.title(f'Frame {frame_idx}')
    plt.axis('off')

    # Plot the histogram
    plt.subplot(1, 2, 2)
    plt.title(f'Histogram for Frame {frame_idx}')
    plt.xlabel('Bins')
    plt.ylabel('Pixels')
    colors = ('r', 'g', 'b')
    for i, color in enumerate(colors):
        plt.plot(hist[i], color=color)
        plt.xlim([0, 256])

    if filename:
        plt.savefig(filename)
    else:
        plt.show()
    plt.close()
def ICV_process_all_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = {}
    histograms = {}

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hist = ICV_histogram(frame_rgb)
        frames[frame_idx] = frame_rgb
        histograms[frame_idx] = hist
        frame_idx += 1

    cap.release()
    return frames, histograms

# Process and store all frames and histograms
frames, histograms = ICV_process_all_frames('DatasetB.avi')


In [ ]:
#Display the selected frame along with histogram
ICV_histogram_with_frame(frames[11], histograms[11], 11)

In [ ]:
ICV_histogram_with_frame(frames[100], histograms[100], 100)

Write a function that returns the value of the intersection of a pair of histograms. For a given video
sequence, use the histogram intersection function to calculate the intersection between consecutive
frames (e.g. between It and It+1, between It+1 and It+2 and so on). Find how to normalize the
intersection. Does that change the results? Plot the intersection values over time and the normalised
intersection values, and save the corresponding figures. Show and comment the figures in the report.


In [ ]:
# Process and store all frames and histograms
def ICV_histogram_intersection(hist1, hist2):
    intersection = 0
    for channel in range(3):
        intersection += np.sum(np.minimum(hist1[channel], hist2[channel]))
    return intersection

# Function to plot the overall histogram intersection for two specific frames
def ICV_plot_histogram_intersections(frame_idx, histograms, filename):
    plt.figure(figsize=(12, 4))
    colors = ['r', 'g', 'b']
    for channel in range(3):
        plt.subplot(1, 3, channel+1)
        plt.bar(range(256), np.minimum(histograms[frame_idx][channel], histograms[frame_idx+1][channel]), color=colors[channel], alpha=0.7)
        plt.title(f'Intersection {colors[channel].upper()} Channel')
        plt.xlabel('Bin')
        plt.ylabel('Frequency')
    plt.suptitle(f'Histogram Intersections: Frames {frame_idx} and {frame_idx+1}')
    plt.savefig(filename)
    plt.show()
ICV_histogram_with_frame(frames[300], histograms[300], 300)
ICV_histogram_with_frame(frames[301], histograms[301], 301)
# Plot histogram intersections for specific frames
ICV_plot_histogram_intersections(300, histograms, 'pair_histogram_intersection.png')


# Calculate intersection values for the entire video sequence
intersection_values = []
normalized_intersection_values = []
for i in range(len(histograms) - 1):
    intersection = ICV_histogram_intersection(histograms[i], histograms[i+1])
    intersection_values.append(intersection)
    total_count = sum([np.sum(histograms[i][channel]) for channel in range(3)])
    normalized = intersection / total_count
    normalized_intersection_values.append(normalized)

# Plotting function for raw and normalized intersection values
def plot_intersection(values, title, filename):
    plt.figure(figsize=(10, 5))
    plt.plot(values)
    plt.xlabel('Frame Index')
    plt.ylabel('Intersection Value')
    plt.title(title)
    plt.savefig(filename)
    plt.show()

# Plot and save the raw intersection values
plot_intersection(intersection_values, 'Raw Histogram Intersection', 'raw_intersection_video.png')

# Plot and save the normalized intersection values
plot_intersection(normalized_intersection_values, 'Normalized Histogram Intersection ', 'normalized_intersection_video.png')


4) Texture Classification. (Use Datasets A and B)
The Local Binary Pattern (LBP) operator describes the surroundings of a pixel by generating a bit-code
from the binary derivatives of a pixel.

a) Write a function that divides a greyscale image into equally sized non-overlapping windows and
returns the feature descriptor for each window as distribution of LBP codes. For each pixel in the
window, compare the pixel to each of its 8 neighbours. Convert the resulting bit-codes (base 2) to
decimals (base 10 numbers) and compute their histogram over the window. Normalize the histogram
(which is now a feature descriptor representing the window). Show in the report the resulting images.

In [ ]:
def ICV_lbp(image, window_size):
    """
    This function divides the image into non-overlapping windows,
    applies the LBP operation on each window, and computes the normalized
    histogram of LBP codes for each window.
    """
    rows, cols = image.shape
    lbp_image = np.zeros_like(image, dtype=np.uint8)
    # Determine the number of windows in each dimension
    num_windows_x = cols // window_size
    num_windows_y = rows // window_size
    # a list histograms for each window
    histograms = []
    # Process each window
    for win_y in range(0, rows - window_size + 1, window_size):
        for win_x in range(0, cols - window_size + 1, window_size):
            window = image[win_y:win_y + window_size, win_x:win_x + window_size]
            lbp_window = np.zeros_like(window, dtype=np.uint8)

            # LBP operation per window
            for i in range(1, window_size - 1):
                for j in range(1, window_size - 1):
                    center = window[i, j]
                    binary_str = ''
                    # Compare with the 8 neighbors to create the binary string
                    for dy, dx in [(-1, -1), (-1, 0), (-1, 1),
                                   (0, 1), (1, 1), (1, 0),
                                   (1, -1), (0, -1)]:
                        binary_str += '1' if window[i + dy, j + dx] > center else '0'

                    lbp_window[i, j] = int(binary_str, 2)

            # Update the LBP image
            lbp_image[win_y:win_y + window_size, win_x:win_x + window_size] = lbp_window

            # Calculate and normalize histogram for the window
            hist, _ = np.histogram(lbp_window.ravel(), bins=np.arange(257), range=(0, 256))
            hist = hist.astype("float")
            hist /= hist.sum()
            histograms.append(hist)
    return histograms, lbp_image

window_size = 32
img = cv2.imread('car-1.jpg', cv2.IMREAD_GRAYSCALE)
# Compute the LBP for windowed image
histograms, lbp_img = ICV_lbp(img, window_size)

In [ ]:
def ICV_visualize_selected_windows(image, lbp_image, histograms, window_size, selected_indices):
    for idx in selected_indices:
        # Calculate the position of the window
        num_windows_x = image.shape[1] // window_size
        x = (idx % num_windows_x) * window_size
        y = (idx // num_windows_x) * window_size

        # Extract the window from the original and LBP images
        window = image[y:y + window_size, x:x + window_size]
        lbp_window = lbp_image[y:y + window_size, x:x + window_size]

        # Display the original window
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(window, cmap='gray')
        plt.title(f'Original Window {idx+1}')
        plt.axis('off')

        # Display the LBP of the window
        plt.subplot(1, 3, 2)
        plt.imshow(lbp_window, cmap='gray')
        plt.title(f'LBP Window {idx+1}')
        plt.axis('off')

        # Display the histogram for the window
        plt.subplot(1, 3, 3)
        plt.bar(range(256), histograms[idx])
        plt.title(f'Normalized LBP Histogram {idx+1}')
        plt.tight_layout()
        plt.show()

#  window indices
selected_indices = [0, 2, 35]
ICV_visualize_selected_windows(img, lbp_img, histograms, window_size, selected_indices)


b) Come up with a descriptor that represents the whole image as consisting of multiple windows. For
example, you could combine several local descriptions into a global description by concatenation.
Discuss in the report alternative approaches. Using the global descriptor you created, implement a
classification process that separates the images in the dataset into two categories: face images and
non-face images (for example, you could use histogram similarities). Comment the results in the
report. Is the global descriptor able to represent whole images of different types (e.g. faces vs. cars)?
Identify problems (if any), discuss them in the report and suggest possible solutions.


In [ ]:
#Calculate the Euclidean distance between two histograms
def ICV_euclidean_distance(hist1, hist2):
    return np.sqrt(np.sum((hist1 - hist2) ** 2))

def ICV_classify_image(test_image, training_images, training_labels, window_size):
    # Compute the histogram of the test image using
    test_histograms, _ = ICV_lbp(test_image, window_size)
    # Concatenate histograms to create a descriptor for the test image
    test_descriptor = np.concatenate(test_histograms)

    nearest_label = None
    min_distance = float('inf')
    # Compare the test image descriptor with each training image descriptor
    for train_image, label in zip(training_images, training_labels):
        train_histograms, _ = ICV_lbp(train_image, window_size)
        train_descriptor = np.concatenate(train_histograms)
        # Calculate the Euclidean distance between descriptors
        distance = ICV_euclidean_distance(test_descriptor, train_descriptor)

        if distance < min_distance:
            min_distance = distance
            nearest_label = label

    return nearest_label,test_descriptor

# Reading the images in grayscale
car = cv2.imread('car-1.jpg', cv2.IMREAD_GRAYSCALE)
car2 = cv2.imread('car-2.jpg', cv2.IMREAD_GRAYSCALE)
face = cv2.imread('face-1.jpg', cv2.IMREAD_GRAYSCALE)
face2 = cv2.imread('face-2.jpg', cv2.IMREAD_GRAYSCALE)
test_face = cv2.imread('face-3.jpg', cv2.IMREAD_GRAYSCALE)
test_car = cv2.imread('car-3.jpg', cv2.IMREAD_GRAYSCALE)

training_images = [car, car2, face, face2]
training_labels = ['non-face', 'non-face', 'face', 'face']

window_size=50
# Classifying test images
car_test,test_descriptor = ICV_classify_image(test_car, training_images, training_labels, window_size)
face_test,test_descriptor2 = ICV_classify_image(test_face, training_images, training_labels, window_size)
# Printing results
print("Car test lebel:",car_test)
print("Face test lebel:",face_test)

In [ ]:
def ICV_plot_descriptor_histogram(descriptor, title="Descriptor Histogram"):
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(descriptor)), descriptor, width=1.0)
    plt.title(title)
    plt.xlabel('Descriptor Bin')
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
ICV_plot_descriptor_histogram(test_descriptor, "Test car descriptor histogram")
ICV_plot_descriptor_histogram(test_descriptor2, "Test face descriptor histogram")

c) Decrease the window size and perform classification again. Comment the results in the report.

In [ ]:
# Decreased window size
window_size = 30

# Classifying test images with the new window size
car_test1, test_descriptor3 = ICV_classify_image(test_car, training_images, training_labels, window_size)
face_test1, test_descriptor4 = ICV_classify_image(test_face, training_images, training_labels, window_size)

# Printing results
print("Car test label:", car_test1)
print("Face test label:", face_test1)

# Plotting descriptor histograms for the test images
ICV_plot_descriptor_histogram(test_descriptor3, "Test car descriptor histogram")
ICV_plot_descriptor_histogram(test_descriptor4, "Test face descriptor histogram")


d) Increase the window size and perform classification again. Comment the results in the report.

In [ ]:
# Decreased window size
window_size = 100

# Classifying test images with the new window size
car_test2, test_descriptor5 = ICV_classify_image(test_car, training_images, training_labels, window_size)
face_test2, test_descriptor6 = ICV_classify_image(test_face, training_images, training_labels, window_size)

# Printing results
print("Car test label:", car_test2)
print("Face test label:", face_test2)

# Plotting descriptor histograms for the test images
ICV_plot_descriptor_histogram(test_descriptor5, "Test car descriptor histogram")
ICV_plot_descriptor_histogram(test_descriptor6, "Test face descriptor histogram")


e) Discuss how LBP can be used or modified for the analysis of dynamic textures in a video.

Moving objects captured by fixed cameras are the focus of several computer vision applications.

a) Write a function that performs pixel-by-pixel frame differencing using, as reference frame, the first
frame of an image sequence. Apply a classification threshold and save the results.

In [ ]:
def ICV_pixel_thresholding(frame, threshold):
    """
    Apply thresholding to a frame pixel by pixel.
    :param frame: Grayscale image frame.
    :param threshold: Threshold value.
    :return: Thresholded frame.
    """
    height, width = frame.shape
    thresholded = np.zeros((height, width), dtype=np.uint8)

    for h in range(height):
        for w in range(width):
            thresholded[h, w] = 255 if frame[h, w] > threshold else 0

    return thresholded

def ICV_frame_difference(reference, current):
    """
    Compute the absolute difference between two frames pixel by pixel.
    Stores the original frames, frame differencing results, and threshold results in dictionaries.
    :param reference: Reference grayscale frame.
    :param current: Current grayscale frame.
    """
    height, width = reference.shape
    diff_frame = np.zeros((height, width), dtype=np.uint8)

    for i in range(height):
        for j in range(width):
            diff = abs(int(current[i, j]) - int(reference[i, j]))
            diff_frame[i, j] = diff

    return diff_frame

def ICV_process_video(video_path):
    cap = cv2.VideoCapture(video_path)

    # Read the first frame as the reference frame
    _, reference_frame = cap.read()
    reference_frame_gray = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)

    # Dictionaries to store frames, differences, and thresholds
    frames, differencing, thresholded = {}, {}, {}

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Compute the absolute difference and apply threshold
        difference_frame = ICV_frame_difference(reference_frame_gray, current_frame_gray)
        thresholded_frame = ICV_pixel_thresholding(difference_frame, 30)

        # Store frames and results
        frames[frame_count] = current_frame_gray
        differencing[frame_count] = difference_frame
        thresholded[frame_count] = thresholded_frame

        frame_count += 1

    cap.release()
    return frames, differencing, thresholded, reference_frame_gray

# Processing the video
video_path = 'DatasetC.avi'
frames, diff, thresholded, reference_frame_gray = ICV_process_video(video_path)

plt.figure(figsize=(12, 8))
# Display the reference frame
plt.subplot(3, 4, 1)
plt.imshow(reference_frame_gray, cmap='gray')
plt.title("Reference Frame")

indices_to_display = [10, 100]

for i, idx in enumerate(indices_to_display):
    # Original Frame
    plt.subplot(3, 4, i * 4 + 2)
    plt.imshow(frames[idx], cmap='gray')
    plt.title(f"Original Frame {idx}")

    # Difference Frame
    plt.subplot(3, 4, i * 4 + 3)
    plt.imshow(diff[idx], cmap='gray')
    plt.title(f"Diff Frame {idx}")

    # Thresholded Frame
    plt.subplot(3, 4, i * 4 + 4)
    plt.imshow(thresholded[idx], cmap='gray')
    plt.title(f"Threshold Frame {idx}")

plt.tight_layout()
plt.show()


b) Repeat the exercise using the previous frame as reference frame (use frame It-1 as reference frame
for frame It, for each t). Comment the results in the report.

In [ ]:
def ICV_frame_differencing(video_path, threshold):
    """
    Performs frame differencing using the previous frame as reference.
    Stores the original frames, frame differencing results, and threshold results in dictionaries.

    :param video_path: Path to the input video file.
    :param threshold: Classification threshold for differencing.
    """
    cap = cv2.VideoCapture(video_path)

    # Check if the video is opened successfully
    if not cap.isOpened():
        print("Error opening video file")
        return

    # Initialize dictionaries to store frames, differences, and thresholds
    frames, diffs, thresholds = {}, {}, {}

    # Read the first frame and set it as the previous frame
    ret, prev_frame = cap.read()
    if not ret:
        print("Failed to read the video.")
        return

    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0

    while True:
        ret, current_frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

        # Compute the absolute difference
        diff = ICV_frame_difference(prev_frame_gray, current_frame_gray)

        # Apply threshold
        thresh = ICV_pixel_thresholding(diff, threshold)

        # Store the frames, differences, and thresholds
        frames[frame_count] = current_frame_gray
        diffs[frame_count] = diff
        thresholds[frame_count] = thresh

        # Update the previous frame
        prev_frame_gray = current_frame_gray
        frame_count += 1

    cap.release()
    return frames, diffs, thresholds

video_path = 'DatasetC.avi'
threshold = 50
frames, diffs, thresholds = ICV_frame_differencing(video_path, threshold)

# Displaying selected frames along with their difference and thresholded frames
plt.figure(figsize=(12, 8))
# Select indices to display
indices_to_display = [10, 100]

for i, idx in enumerate(indices_to_display):
    # Original Frame
    plt.subplot(3, 3, i * 3 + 1)
    plt.imshow(frames[idx], cmap='gray')
    plt.title(f"Original Frame {idx}")

    # Difference Frame
    plt.subplot(3, 3, i * 3 + 2)
    plt.imshow(diffs[idx], cmap='gray')
    plt.title(f"Diff Frame {idx}")

    # Thresholded Frame
    plt.subplot(3, 3, i * 3 + 3)
    plt.imshow(thresholds[idx], cmap='gray')
    plt.title(f"Threshold Frame {idx}")
plt.tight_layout()
plt.show()

c) Write a function that generates a reference frame (background) for the sequence using for example
frame differencing and a weighted temporal averaging algorithm.

In [ ]:
def ICV_weighted_average(background, current_frame, alpha):
    # Apply temporal weighting to update the background
    return (1 - alpha) * background + alpha * current_frame

def ICV_background(video_path, base_alpha=0.01, max_alpha=0.1, threshold=10, movement_threshold=50):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video stream or file")
        return None

    ret, background_frame = cap.read()
    if not ret:
        print("Could not read the first frame.")
        cap.release()
        return None

    # Initialize the background
    background = cv2.cvtColor(background_frame, cv2.COLOR_BGR2GRAY).astype(np.float32)
    prev_frame = background.copy()

    while True:
        ret, current_frame = cap.read()
        if not ret:
            break

        current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY).astype(np.float32)
        # Compute the frame difference with the previous frame
        diff = ICV_frame_difference(prev_frame, current_frame_gray)

        # Apply thresholding to get the moving objects mask
        movement = ICV_pixel_thresholding(diff, movement_threshold)

        # Invert the mask background areas are 1 moving objects are 0
        background_mask = 1 - (movement / 255).astype(np.float32)

        # Calculate dynamic alpha for background areas
        alpha = ICV_weighted_average(base_alpha, max_alpha, background_mask)

        # Update the background only where there is no movement
        background = (1 - alpha) * background + alpha * current_frame_gray * background_mask

        prev_frame = current_frame_gray

    cap.release()
    return background.astype(np.uint8)

video_path = 'DatasetC.avi'
background = ICV_background(video_path)

# Save the generated reference frame
cv2.imwrite("background.png", background)